In [1]:
import ee
import geemap

In [29]:
# 显示地图，并选中湖泊
Map = geemap.Map(center=[40.964,108.884],zoom = 10)
Map

Map(center=[40.964, 108.884], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [58]:
#在地图中选择研究区，并转为ROI。选定好后再运行下两行代码
Map.draw_last_feature
roi = ee.FeatureCollection(Map.draw_last_feature)

In [59]:
l8_vis_params = {
  'bands':['B5','B4','B3'],
  'min':0,
  'max': 0.3
}

In [72]:
# 选择L8-SR影像
l8_col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi)\
    .filter(ee.Filter.calendarRange(2019,2019,'year'))\
    .filter(ee.Filter.calendarRange(7,10,'month'))\

print(f"""已选择{l8_col.size().getInfo()}景L8影像""")
print(l8_col.aggregate_array('system:index').getInfo())
print(l8_col.aggregate_array('CLOUD_COVER').getInfo())

已选择14景L8影像
['LC08_125032_20190714', 'LC08_125032_20190730', 'LC08_125032_20190815', 'LC08_125032_20190831', 'LC08_125032_20190916', 'LC08_125032_20191002', 'LC08_125032_20191018', 'LC08_126032_20190705', 'LC08_126032_20190721', 'LC08_126032_20190806', 'LC08_126032_20190907', 'LC08_126032_20190923', 'LC08_126032_20191009', 'LC08_126032_20191025']
[2.03, 3.19, 3.63, 3.55, 1.53, 0.17, 40.26, 62.31, 27.27, 5.2, 18.29, 11.5, 41.48, 4.44]


In [79]:
def l8_sr_select_by_cloud(l8_col,roi,threshold):
    ##依次传入影像集合、研究区roi，云量阈值
    
    #计算l8_collection-Imagecollection中Image数量
    count = l8_col.size().getInfo()
    print(count)
    #定义图像裁剪函数
    def img_clip(image):
        clip_img = image.clip(roi)
        return clip_img        
    
    #进行图像裁剪
    l8_col_clip = l8_col.map(img_clip)
    l8_col_clip_list = l8_col_clip.toList(count)
    #创建列表，用来存储满足
    nice_list = []
    
    #qa波段云量识别
    def rmCloud(image):
        cloudShadowBitMask = (1 << 3)
        cloudsBitMask = (1 << 5)
        qa = image.select("pixel_qa")
        mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
        return mask   
    
    #定义云量计算函数
    def l8_img_cloud_cal(image):
        mask = rmCloud(image)
        hist = mask.reduceRegion(ee.Reducer.histogram(1), roi, 30).getInfo()
        #print(hist['pixel_qa']['histogram'])
        if len(hist['pixel_qa']['histogram']) == 2:
            count_clear = hist['pixel_qa']['histogram'][1]
            count_sum = hist['pixel_qa']['histogram'][0]+hist['pixel_qa']['histogram'][1]
            print(f"""{image.getInfo()['properties']['system:index']}研究区云量为：{1-(count_clear/count_sum)},全景云量为:{image.getInfo()['properties']['CLOUD_COVER']/100},
                被选中：{(1-(count_clear/count_sum))<threshold}""")
            if((1-(count_clear/count_sum))<threshold):
                nice_list.append('LANDSAT/LC08/C01/T1_TOA/'+image.getInfo()['properties']['system:index'])
        elif len(hist['pixel_qa']['histogram']) == 1:
            count_sum = hist['pixel_qa']['histogram'][0]
            nice_list.append('LANDSAT/LC08/C01/T1_TOA/'+image.getInfo()['properties']['system:index'])
        #return image
    
    #遍历Image，计算云量，获取满足条件的影像。
    for i in range(count):
        img = ee.Image(l8_col_clip_list.get(i))
        print(f"""第{i+1}景影像处理中，未提示为研究区内无云""")
        l8_img_cloud_cal(img)
    #print(nice_list)
    
    #将选择的影像与少云影像集合合并
    selected_col = ee.ImageCollection(nice_list)
    #result_col = cloudless_col.merge(selected_col)
    
    return selected_col
    
    

In [80]:
result_col = l8_sr_select_by_cloud(l8_col,roi,0.1)

print(f"""共{result_col.size().getInfo()}景L8研究区无云影像""")
print(result_col.aggregate_array('system:index').getInfo())

14
第1景影像处理中，未提示为研究区内无云
LC08_125032_20190714研究区云量为：0.010022092214348244,全景云量为:0.0203,
                被选中：True
第2景影像处理中，未提示为研究区内无云
LC08_125032_20190730研究区云量为：0.037274362622617296,全景云量为:0.0319,
                被选中：True
第3景影像处理中，未提示为研究区内无云
LC08_125032_20190815研究区云量为：0.00011314356921843238,全景云量为:0.0363,
                被选中：True
第4景影像处理中，未提示为研究区内无云
LC08_125032_20190831研究区云量为：0.03049193617489132,全景云量为:0.0355,
                被选中：True
第5景影像处理中，未提示为研究区内无云
LC08_125032_20190916研究区云量为：4.335733177440737e-05,全景云量为:0.015300000000000001,
                被选中：True
第6景影像处理中，未提示为研究区内无云
LC08_125032_20191002研究区云量为：0.0005030670494809986,全景云量为:0.0017000000000000001,
                被选中：True
第7景影像处理中，未提示为研究区内无云
LC08_125032_20191018研究区云量为：0.5552350389475289,全景云量为:0.40259999999999996,
                被选中：False
第8景影像处理中，未提示为研究区内无云
LC08_126032_20190705研究区云量为：0.8454217582517984,全景云量为:0.6231,
                被选中：False
第9景影像处理中，未提示为研究区内无云
LC08_126032_20190721研究区云量为：0.0031740601640650734,全景云量为:0.2727,
               